In [1]:
import os
import dotenv

dotenv.load_dotenv(".env")

True

In [2]:
from utils import get_algod_client
from account import Account

client = get_algod_client(os.environ.get("ALGOD_URL"), os.environ.get("ALGOD_API_KEY"))
creator = Account.from_mnemonic(os.environ.get("CREATOR_MNEMONIC"))

creator.get_address()

'U22M6ENJLHAQEIL6TVCC3BJFM63JLFVH4ZFS3XYPA3XRUCW3NN5MKC5YVU'

In [3]:
from typing import Tuple

from base64 import b64decode
from algosdk.future import transaction
from algosdk.v2client.algod import AlgodClient
from algosdk.logic import get_application_address
from pyteal import compileTeal, Mode, Expr

from contracts.validator import AllyValidator
from utils import wait_for_transaction


def fullyCompileContract(client: AlgodClient, contract: Expr) -> bytes:
    teal = compileTeal(contract, mode=Mode.Application, version=5)
    response = client.compile(teal)
    return b64decode(response["result"])


def get_contracts(client: AlgodClient) -> Tuple[bytes, bytes]:
    simple_contract = AllyValidator()

    approval_program = fullyCompileContract(
        client, simple_contract.approval_program())
    clear_state_program = fullyCompileContract(
        client, simple_contract.clear_state_program())

    return approval_program, clear_state_program


def create_pool(client: AlgodClient, creator: Account):
    approval, clear = get_contracts(client)
    global_schema = transaction.StateSchema(num_uints=7, num_byte_slices=2)
    local_schema = transaction.StateSchema(num_uints=0, num_byte_slices=0)

    txn = transaction.ApplicationCreateTxn(
        sender=creator.get_address(),
        sp=client.suggested_params(),
        on_complete=transaction.OnComplete.NoOpOC,
        approval_program=approval,
        clear_program=clear,
        global_schema=global_schema,
        local_schema=local_schema,
    )
    signed_txn = txn.sign(creator.get_private_key())
    client.send_transaction(signed_txn)

    response = wait_for_transaction(client, signed_txn.get_txid())
    assert response.application_index is not None and response.application_index > 0
    return response.application_index


app_id = create_pool(client, creator)

print(f"App ID: {app_id}")
print(f"App address: {get_application_address(app_id)}")

Waiting for confirmation...
Transaction 5CXJJ4JFBRXDQPHVJVD7OXH4JBRPVCPJDLOVBDPRWHQCXH6KJB5Q confirmed in round 9043.
App ID: 7
App address: PQ4NBMDLREVVRUQUFX7CFEUTKEBCEJOQ5YREC6SZA4EYKOFKHO2VLDIMTQ


In [4]:
def bootstrap_pool(client: AlgodClient, sender: Account, application_id: int):
    txn = transaction.ApplicationCallTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        index=application_id,
        on_complete=transaction.OnComplete.NoOpOC,
        app_args=[
            b"bootstrap"
        ]
    )
    signed_txn = txn.sign(sender.get_private_key())
    client.send_transaction(signed_txn)

    wait_for_transaction(client, signed_txn.get_txid())
    

bootstrap_pool(client, creator, app_id)

Waiting for confirmation...
Transaction JEAJAWYVETXS3QG4XFRPPN65BVCQXQNPWDB2OAZUBDURK53UIOEQ confirmed in round 9070.


In [6]:
from utils import get_app_global_state

get_app_global_state(client, app_id)

{b'token_id': 10}